## PROJ - notebook

Un  `.ipynb` pour faire tourner les tests et écrire les résultats.

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));
include(joinpath("src", "resolution.jl"))
include(joinpath("src", "cutting_planes.jl"));
include(joinpath("src", "branch_and_cut.jl"));
include(joinpath("src", "dual_solve.jl"));
include(joinpath("src", "heuristics.jl"));
include(joinpath("src", "testing.jl"));
include(joinpath("src", "latex.jl"));
include(joinpath("src", "graphics.jl"));

Définit les méthodes à tester et les fichier dans lesquels mettre les outputs :

In [ ]:
statique = completeModelWrapper(resolution, time_budget=2 * 60.0)

dual = completeModelWrapper(dualSolve, time_budget=2 * 60.0)

time_budget = nSquaredTimeWithMax(maxtime=10 * 60.0, mintime=10.0)
heuristique = heurWrapper(time_budget_fun=time_budget)

cutting_planes = completeModelWrapper(plans_coupants, time_budget=2 * 60.0);

b_and_c = completeModelWrapper(branch_and_cut, time_budget=2 * 60.0);

In [ ]:
statique_file = joinpath("demo_data", "results", "statique.json")

dual_file = joinpath("demo_data", "results", "dual.json")

heuristique_file = joinpath("demo_data", "results", "heuristique.json")

cutting_plane_file = joinpath("demo_data", "results", "cutting_plane.json")

branch_and_cut_file = joinpath("demo_data", "results", "branch_and_cut.json")

instance_dir = joinpath("demo_data", "instances");

Tests des différentes méthodes :

In [ ]:
testMethod(
    method=dual;
    method_name="Dualisation",
    save=dual_file,
    instance_dir=instance_dir,
)

In [ ]:
testMethod(
    method=heuristique;
    method_name="Heuristique",
    save=heuristique_file,
    instance_dir=instance_dir,
)

In [ ]:
testMethod(
    method=cutting_planes;
    method_name="Plans coupants",
    save=cutting_plane_file,
    instance_dir=instance_dir,
)

In [ ]:
testMethod(
    method=b_and_c;
    method_name="Branch & Cut",
    save=branch_and_cut_file,
    instance_dir=instance_dir,
)

In [ ]:
testMethod(
    method=statique;
    method_name="Statique",
    save=statique_file,
    instance_dir=instance_dir,
)

Exploitation des tests (tableau de résultats, diagramme de performance, solutions + valeurs) :

In [ ]:
res_files = [
    statique_file,
    dual_file,
    heuristique_file,
    cutting_plane_file,
    branch_and_cut_file,
];

In [ ]:
processed = processResultsForTable(
    res_files;
    not_robust_method="Statique",
)
output_file = joinpath("demo_out", "demo_table.tex")

open(
    fd -> writeTable(
        ["Dualisation", "Plans coupants", "Branch & Cut", "Heuristique"];
        results=processed,
        template_file=joinpath("template", "table.tex"),
        output=fd,
    ),
    output_file,
    write=true,
)

In [ ]:
graphic_file = joinpath("demo_out", "demo_perf_diag.png")
graphic_results = processResultsForPerfDiag(
    res_files;
    not_robust_method="Statique",
)
savePerfDiag(
    graphic_results;
    until=300.0,
    output=graphic_file,
)

In [ ]:
sols_file = joinpath("demo_out", "demo_solutions.tex")
sols_res = processResultsForSolutions(
    res_files;
    not_robust_method="Statique",
)
open(
    fd -> writeSolutions(
        sols_res;
        template_file=joinpath("template", "solutions.tex"),
        output=fd,
    ),
    sols_file,
    write=true,
)